In [ ]:
pip install transformers datasets torch

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, AdamW
from datasets import load_dataset
import torch

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

squad = load_dataset("squad")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples['question']]
    inputs = tokenizer(
        questions,
        examples['context'],
        max_length=384,
        truncation=True,
        return_offsets_mapping=True,
        padding="max_length",
        return_tensors="pt"
    )
    start_positions = []
    end_positions = []

    for i, offset_mapping in enumerate(inputs["offset_mapping"]):
        answer = examples["answers"][i]["text"][0]
        start_char = examples["answers"][i]["answer_start"][0]
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        start_position = context_start
        end_position = context_end

        for idx, mapping in enumerate(offset_mapping):
            if mapping[0] <= start_char < mapping[1]:
                start_position = idx
            if mapping[0] < end_char <= mapping[1]:
                end_position = idx
                break

        start_positions.append(start_position)
        end_positions.append(end_position)

    inputs["start_positions"] = torch.tensor(start_positions)
    inputs["end_positions"] = torch.tensor(end_positions)
    return inputs
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataset = tokenized_squad["train"].select(range(1000))
val_dataset = tokenized_squad["validation"].select(range(100))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=data_collator)


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(3):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != "offset_mapping"}

        if "start_positions" not in inputs or "end_positions" not in inputs:
            continue

        outputs = model(**inputs)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Loss: 6.008988380432129
Loss: 5.8429789543151855
Loss: 5.704751014709473
Loss: 5.600870132446289
Loss: 5.44846248626709
Loss: 5.400055408477783
Loss: 5.24246883392334
Loss: 5.124034881591797
Loss: 5.22344970703125
Loss: 5.020404815673828
Loss: 4.910091400146484
Loss: 4.616632461547852
Loss: 4.789853096008301
Loss: 4.766843795776367
Loss: 5.0531005859375
Loss: 4.385164260864258
Loss: 4.85910177230835
Loss: 4.167695045471191
Loss: 4.408359527587891
Loss: 4.317964553833008
Loss: 4.076933860778809
Loss: 4.80088996887207
Loss: 4.552515983581543
Loss: 4.155620574951172
Loss: 4.285686492919922
Loss: 4.014763832092285
Loss: 3.939174175262451
Loss: 4.582718849182129
Loss: 4.13530158996582
Loss: 4.446966171264648
Loss: 4.668543815612793
Loss: 4.365328788757324
Loss: 3.6279659271240234
Loss: 4.430480480194092
Loss: 3.3777153491973877
Loss: 4.108795166015625
Loss: 4.193292617797852
Loss: 3.878500461578369
Loss: 3.813345432281494
Loss: 3.687178373336792
Loss: 3.534700870513916
Loss: 3.8910150527954

In [ ]:
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt', max_length=384, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)

    answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end+1], skip_special_tokens=True)
    return answer



In [ ]:
context = input("Enter passage:")
question = input("Enter question:")

print(answer_question(question, context))

Enter passage:Artificial Intelligence (AI) is a field of computer science that focuses on creating intelligent machines that work and react like humans. AI is used in a variety of industries including healthcare, finance, and autonomous vehicles.The Great Wall of China is a series of fortifications that were built across the historical northern borders of China to protect against various nomadic groups. Construction of the wall began as early as the 7th century BC and continued for centuries.Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that can later be released to fuel the organisms' activities. This process is essential for the production of oxygen and takes place mainly in the chloroplasts of plant cells.Mount Everest is the Earth's highest mountain, located in the Himalayas. Its peak rises to an elevation of 8,848 meters (29,029 feet) above sea level. It attracts many climbers, including highly experienced mountaineers 